## Token Classification Inference using Online Endpoints

This sample shows how to deploy `token-classification` type models to an online endpoint for inference.

### Task
`token-classification` assigns a label to individual tokens in a sentence. One of the most common `token-classification` tasks is Named Entity Recognition (NER). NER attempts to find a label for each entity in a sentence, such as a person, location, or organization.

### Model
Models that can perform the `token-classification` task are tagged with `task: token-classification`. We will use the `Jean-Baptiste-camembert-ner` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name. If you don't find a model that suits your scenario or domain, you can discover and [import models from HuggingFace hub](../../import/import-model-from-huggingface.ipynb) and then use them for inference. 

### Inference data
We will use the [Jean-Baptiste/wikiner_fr](https://huggingface.co/datasets/Jean-Baptiste/wikiner_fr) dataset. A copy of this dataset is available in the [Jean-Baptiste-wikiner_fr](./Jean-Baptiste-wikiner_fr/) folder. \
Please note that the dataset used here is a French dataset, as the Jean-Baptiste/camembert-ner model was trained in French.

### Outline
* Set up pre-requisites.
* Pick a model to deploy.
* Prepare data for inference. 
* Deploy the model for real time inference.
* Test the endpoint
* Clean up resources.

### 1. Set up pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [1]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    ClientSecretCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
    credential,
    subscription_id="ea4faa5b-5e44-4236-91f6-5483d5b17d14",
    resource_group_name="amyharrispersonal",
    workspace_name="amyharris-canary",
)
# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml-preview"
registry_ml_client = MLClient(credential, registry_name="azureml-preview")

Class FeatureStoreOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureSetOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureStoreEntityOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `token-classification` task. In this example, we use the `Jean-Baptiste-camembert-ner` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

In [2]:
model_name = "Jean-Baptiste-camembert-ner"
model_version = "3"
foundation_model = registry_ml_client.models.get(model_name, model_version)
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(
        foundation_model.name, foundation_model.version, foundation_model.id
    )
)



Using model name: jean-baptiste-camembert-ner, version: 3, id: azureml://registries/azureml-preview/models/jean-baptiste-camembert-ner/versions/3 for inferencing


### 3. Prepare data for inference.

A subset of the Jean-Baptiste-wikiner_fr dataset is available in the [Jean-Baptiste-wikiner_fr](./Jean-Baptiste-wikiner_fr/) folder.  The next few cells show basic data preparation:
* Visualize some data rows
* Save few samples in the format that can be passed as input to the online-inference endpoint.

In [3]:
# load the ./Jean-Baptiste-wikiner_fr/train_100.jsonl file into a pandas dataframe and show the first 5 rows
import pandas as pd

pd.set_option(
    "display.max_colwidth", 0
)  # set the max column width to 0 to display the full text
train_df = pd.read_json("./Jean-Baptiste-wikiner_fr/train_100.jsonl", lines=True)
train_df.head()

,id,tokens,ner_tags,text,ner_tags_str
0,10946,"[L', agglomération, à, mis, en, place, des, voies, supplémentaires, à, Grenoble-Universités-Gières, (, réalisé, en, 2007, ), ,, pour, disposer, à, l', avenir, d', un, réseau, de, trains, de, banlieue, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","L' agglomération à mis en place des voies supplémentaires à Grenoble-Universités-Gières ( réalisé en 2007 ) , pour disposer à l' avenir d' un réseau de trains de banlieue .","[O, O, O, O, O, O, O, O, O, O, LOC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,76495,"[Sous, contrôle, communiste, mais, pour, la, première, fois, autonome, ,, le, régime, de, la, Macédoine, yougoslave, cultive, l', identité, macédonienne, à, partir, de, 1945, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Sous contrôle communiste mais pour la première fois autonome , le régime de la Macédoine yougoslave cultive l' identité macédonienne à partir de 1945 .","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, LOC, O, O, O, O, O, O, O, O, O, O]"
2,41771,"[Dans, son, traité, de, 1600, De, Magnete, ,, le, médecin, anglais, William, Gilbert, forge, le, mot, bas-latin, electricus, ,, pour, désigner, cette, propriété, d', attirer, les, petits, objets, après, frottement, .]","[0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Dans son traité de 1600 De Magnete , le médecin anglais William Gilbert forge le mot bas-latin electricus , pour désigner cette propriété d' attirer les petits objets après frottement .","[O, O, O, O, O, PER, PER, O, O, O, O, PER, PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,23658,"[Pour, son, inauguration, ,, Maradona, ,, Pelé, et, Hicham, El, Guerrouj, ont, été, invités, .]","[0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0]","Pour son inauguration , Maradona , Pelé et Hicham El Guerrouj ont été invités .","[O, O, O, O, PER, O, PER, O, PER, PER, PER, O, O, O, O]"
4,68322,"[Il, mourra, à, Québec, le, 25, décembre, 1635, sans, avoir, fini, ses, préparatifs, de, la, fondation, de, Montréal, qui, n', aura, lieu, qu', en, 1642, .]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",Il mourra à Québec le 25 décembre 1635 sans avoir fini ses préparatifs de la fondation de Montréal qui n' aura lieu qu' en 1642 .,"[O, O, O, LOC, O, O, O, O, O, O, O, O, O, O, O, O, O, LOC, O, O, O, O, O, O, O, O]"


### 4. Deploy the model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [4]:
import time, sys
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    OnlineRequestSettings,
)

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "token-classification-" + str(timestamp)
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + foundation_model.name
    + ", for token-classification task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [5]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_DS2_v2",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=60000,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

Instance type Standard_DS2_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint token-classification-1684194167 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


..................................................................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://token-classification-1684194167.eastus2euap.inference.ml.azure.com/score', 'openapi_uri': 'https://token-classification-1684194167.eastus2euap.inference.ml.azure.com/swagger.json', 'name': 'token-classification-1684194167', 'description': 'Online endpoint for jean-baptiste-camembert-ner, for token-classification task', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/resourcegroups/amyharrispersonal/providers/microsoft.machinelearningservices/workspaces/amyharris-canary/onlineendpoints/token-classification-1684194167', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/providers/Microsoft.MachineLearningServices/locations/eastus2euap/mfeOperationsStatus/oe:c76e6446-545b-4141-80f9-e8ad59c471f2:4b0fc57b-7242-4778-9ad5-c9cd74bbf488?api-version=2022-02-01-previ

### 5. Test the endpoint with sample data

We will fetch some sample data from the test dataset and submit to online endpoint for inference. We will then show the display the scored labels alongside the ground truth labels

In [6]:
import json
import os

# read the ./Jean-Baptiste-wikiner_fr/train_100.jsonl file into a pandas dataframe
df = pd.read_json("./Jean-Baptiste-wikiner_fr/train_100.jsonl", lines=True)
# escape single and double quotes in the text column
df["text"] = df["text"].str.replace("'", "\\'").str.replace('"', '\\"')
# pick 1 random row
sample_df = df.sample(1)
# create a json object with the key as "inputs" and value as a list of values from the en column of the sample_df dataframe
sample_json = {"inputs": sample_df["text"].tolist()}
# save the json object to a file named sample_score.json in the ./Jean-Baptiste-wikiner_fr folder
test_json = {"inputs": {"input_string": sample_df["text"].tolist()}}
# save the json object to a file named sample_score.json in the ./Jean-Baptiste-wikiner_fr folder
with open(os.path.join(".", "Jean-Baptiste-wikiner_fr", "sample_score.json"), "w") as f:
    json.dump(test_json, f)
sample_df.head()

,id,tokens,ner_tags,text,ner_tags_str
62,67763,"[Moscou, est, la, ville, la, plus, peuplée, d', Europe, avec, 10,22, millions, d', habitants, intra, muros, en, 2008, et, 14,74, millions, d', habitants, dans, l', aire, urbaine, .]","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Moscou est la ville la plus peuplée d\' Europe avec 10,22 millions d\' habitants intra muros en 2008 et 14,74 millions d\' habitants dans l\' aire urbaine .","[LOC, O, O, O, O, O, O, O, LOC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [7]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file="./Jean-Baptiste-wikiner_fr/sample_score.json",
)
print("raw response: \n", response, "\n")
# convert the json response to a pandas dataframe
response_df = pd.read_json(response)
response_df.head()

raw response: 
 [{"0": "['I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"}] 



,0
0,"['I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"


In [16]:
# compare the predicted labels with the actual labels
num_tokens = len(sample_df["text"].tolist()[0].split())
predicted_labels = ["O"] * num_tokens
for col in response_df.columns:
    prediction = response_df[col].tolist()[0]
    predicted_labels[prediction["index"] - 1] = prediction["entity"]
compare_df = pd.DataFrame(
    {
        "ground_truth_labels": sample_df["ner_tags_str"].tolist(),
        "predicted_labels": [predicted_labels],
    }
)
compare_df.head()

TypeError: string indices must be integers

### 6. Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()